In [ ]:
import glob
import json
import os

import geopandas
import numpy
import pandas
import seaborn

from matplotlib import pyplot as plt
from rasterstats import gen_zonal_stats
from tqdm import tqdm_notebook as tqdm

In [ ]:
def load_config(path='config.json'):
    """Read JSON config file - copy `config.template.json` to `config.json` and fill with details
    """
    with open(path) as fh:
        config = json.load(fh)
    return config

In [ ]:
data_path = load_config()['data']
data_path

## Count cells developed/undeveloped/existing per Output Area

This step is slow - approx 45m per scenario. Dump to CSV immediately after, load from CSV for further processing and skip this step if possible.

In [ ]:
stats_by_scenario = {}
for key in ['baseline', 'settlements', 'expansion', 'unplanned']:
    stats_by_scenario[key] = [
        f['properties']
        for f in tqdm(gen_zonal_stats(
            os.path.join(data_path, 'boundaries', 'arc_oa_11.gpkg'), 
            os.path.join(data_path, 'udm-results-20191025', '{}_cell_dev.asc'.format(key)),
            categorical=True,
            geojson_out=True
        ), total=11085)
    ]

In [ ]:
dfs = []
for key, data in stats_by_scenario.items():
    df = pandas.DataFrame(data) \
        .fillna(0) \
        .rename(columns={0:'undeveloped', 1: 'existing', 2: 'developed'})
    df['scenario'] = key
    dfs.append(df)
stats = pandas.concat(dfs)

In [ ]:
stats.to_csv('stats.csv', index=False)

In [ ]:
stats = pandas.read_csv('stats.csv')

In [ ]:
assert len(stats) == 11085*4, len(stats) / 11085
assert len(stats.lad11cd.unique()) == 26, len(stats.lad11cd.unique())

In [ ]:
stats.scenario.unique()

In [ ]:
stats.head()

In [ ]:
stats['final'] = (stats.existing + stats.developed)

In [ ]:
# calculate by lad
by_lad = stats[['scenario', 'lad16cd', 'existing', 'developed', 'final']] \
    .groupby(['scenario', 'lad16cd']) \
    .sum() \
    .reset_index() \
    .rename(columns={'existing': 'existing_lad', 'developed': 'developed_lad', 'final': 'final_lad'})

In [ ]:
statsd = stats.merge(by_lad, on=['scenario', 'lad16cd'])
statsd.head(2)

In [ ]:
# read in dwellings per lad per scenario
dfs = []
for scenario, key in [
        ('baseline', 'baseline'), 
        ('settlements', '1-new-cities'), 
        ('expansion', '2-expansion'), 
        ('unplanned', '0-unplanned')
    ]:
    df = pandas.read_csv(os.path.join(data_path, 'socio-economic-1.0.1', 'arc_dwellings__{}.csv'.format(key))) \
        .rename(columns={'lad_uk_2016': 'lad16cd'}) \
        .drop(columns=['lad16nm'])
    df['scenario'] = scenario
    df = df[df.timestep.isin((2015, 2050)) & df.lad16cd.isin(statsdprop.lad16cd.unique())] \
        .sort_values(by=['scenario','lad16cd', 'timestep'])
    dfs.append(df)
dwellings = pandas.concat(dfs).set_index(['lad16cd','scenario','timestep']).unstack(level=-1).reset_index()
dwellings.columns = ['lad16cd', 'scenario', 'dwellings_lad__initial', 'dwellings_lad__final']
dwellings['dwellings_lad__change'] = dwellings.dwellings_lad__final - dwellings.dwellings_lad__initial
dwellings.head(3)

In [ ]:
# calculate dwellings as proportion of lad, as integer
sd = statsd.merge(dwellings, on=['lad16cd', 'scenario'])
sd['dwellings_oa__initial'] = (
    (sd.existing/sd.existing_lad) * sd.dwellings_lad__initial
).round().astype(int)
sd['dwellings_oa__final'] = (
    sd.dwellings_oa__initial + ((sd.developed/(sd.developed_lad+0.1)) * sd.dwellings_lad__change)
).round().astype(int)
sd['dwellings_oa__final_even'] = (
    (sd.final/sd.final_lad) * sd.dwellings_lad__final
).round().astype(int)
sd

In [ ]:
oa_dwellings = sd[['scenario', 'oa11cd', 'lad11cd', 'lad11nm', 'undeveloped','existing','developed','dwellings_oa__initial','dwellings_oa__final','dwellings_oa__final_even']]
oa_dwellings.head(2)

In [ ]:
oa_dwellings.to_csv(os.path.join(data_path, 'processed', 'oa_dwellings.csv'), index=False)

In [ ]:
oa_dwellings_fat = oa_dwellings.pivot_table(columns='scenario', index=['oa11cd', 'lad11cd', 'lad11nm'])
oa_dwellings_fat.columns = ["{}__{}".format(a, b) for a, b in oa_dwellings_fat.columns]
oa_dwellings_fat.head(2)

In [ ]:
oa_dwellings_fat.to_csv(os.path.join(data_path, 'processed', 'oa_dwellings_fat.csv'))

In [ ]:
oas = geopandas.read_file(os.path.join(data_path, 'boundaries', 'arc_oa_11.gpkg'))

In [ ]:
oas = oas.merge(oa_dwellings_fat.reset_index(), on=['oa11cd', 'lad11cd', 'lad11nm'], validate='one_to_one')

In [ ]:
oas.to_file(os.path.join(data_path, 'processed', 'oas_with_dwellings.gpkg'), driver='GPKG')

### Plotting

In [ ]:
oas_d = geopandas.read_file(os.path.join(data_path, 'processed', 'oas_with_dwellings.gpkg'))

In [ ]:
oas_d.columns

In [ ]:
oas_dm = oas_d[[
    'oa11cd', 'st_areasha', 'geometry',
    'dwellings_oa__initial__baseline', 
    'dwellings_oa__final_even__baseline',  
    'dwellings_oa__final_even__expansion',
    'dwellings_oa__final_even__settlements',
    'dwellings_oa__final_even__unplanned'
]].melt(
    id_vars=['oa11cd', 'st_areasha', 'geometry'], 
    value_vars=[
        'dwellings_oa__initial__baseline', 
        'dwellings_oa__final_even__baseline',  
        'dwellings_oa__final_even__expansion',
        'dwellings_oa__final_even__settlements',
        'dwellings_oa__final_even__unplanned'
    ],
    var_name='scenario',
    value_name='dwellings'
)
oas_dm.head(2)

In [ ]:
oas_dm['dwelling_density'] = oas_dm.dwellings / (oas_dm.st_areasha/1e6)
oas_dm['dwelling_density_sqrt'] = numpy.sqrt(oas_dm.dwellings / (oas_dm.st_areasha/1e6))

In [ ]:
oas_dm.describe()

In [ ]:
def plot_map(data, *args, **kwargs):
    cax = plt.gca()
    ax = data.plot(
        column='dwelling_density', 
        ax=cax,
        linewidth=0.001, 
        edgecolor='white',
        legend=(data[0].scenario == 'dwellings_oa__final_even__unplanned'),  # fixme
        vmin=0,
        vmax=3000  # magic number to avoid over-rescaling axis, roughly twice the 75th percentile density
    )
    ax.set_axis_off()
    return ax

seaborn.set_context("paper", font_scale=4)

g = seaborn.FacetGrid(
    data=oas_dm, 
    col='scenario', 
    height=20, 
    sharex=True, 
    sharey=True,
    col_wrap=2
)
fig = g.map_dataframe(plot_map)
fig.savefig('dwellings_density_legend.png')
fig.savefig('dwellings_density_legend.svg')
fig